In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers, callbacks
from sklearn.model_selection import train_test_split


In [2]:
(x_train_full, y_train_full), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

odd_digits = [1, 3, 5, 7, 9]

# Filter only odd digits
train_filter = np.isin(y_train_full, odd_digits)
test_filter = np.isin(y_test, odd_digits)

x_train_full, y_train_full = x_train_full[train_filter], y_train_full[train_filter]
x_test, y_test = x_test[test_filter], y_test[test_filter]


In [ ]:
# Normalize pixel values
x_train_full = x_train_full.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Reshape to add channel dimension
x_train_full = np.expand_dims(x_train_full, -1)
x_test = np.expand_dims(x_test, -1)

# Convert odd digit labels to class indices 0-4
label_map = {digit: i for i, digit in enumerate(odd_digits)}
y_train_full = np.array([label_map[d] for d in y_train_full])
y_test = np.array([label_map[d] for d in y_test])


In [4]:
x_train, x_val, y_train, y_val = train_test_split(
    x_train_full, y_train_full, test_size=0.15, random_state=42
)


In [5]:
def create_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1), name='conv1'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', name='conv2'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu', name='conv3'),
        layers.Flatten(),
        layers.Dense(128, activation='relu', name='dense1'),
        layers.Dense(5, activation='softmax', name='output')  # 5 classes
    ])
    return model

model = create_model()


In [6]:
optimizer = optimizers.Adagrad(learning_rate=0.003)

model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [7]:
checkpoint_cb = callbacks.ModelCheckpoint(
    'best_model.h5', monitor='val_loss', save_best_only=True, verbose=1
)


In [8]:
history1 = model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(x_val, y_val),
    callbacks=[checkpoint_cb]
)


Epoch 1/10
809/811 [============================>.] - ETA: 0s - loss: 0.4451 - accuracy: 0.8847
Epoch 1: val_loss improved from inf to 0.19839, saving model to best_model.h5
811/811 [==============================] - 31s 37ms/step - loss: 0.4446 - accuracy: 0.8848 - val_loss: 0.1984 - val_accuracy: 0.9351
Epoch 2/10
811/811 [==============================] - ETA: 0s - loss: 0.1510 - accuracy: 0.9514
Epoch 2: val_loss improved from 0.19839 to 0.13323, saving model to best_model.h5
811/811 [==============================] - 30s 37ms/step - loss: 0.1510 - accuracy: 0.9514 - val_loss: 0.1332 - val_accuracy: 0.9591
Epoch 3/10
809/811 [============================>.] - ETA: 0s - loss: 0.1123 - accuracy: 0.9651
Epoch 3: val_loss did not improve from 0.13323
811/811 [==============================] - 29s 36ms/step - loss: 0.1125 - accuracy: 0.9651 - val_loss: 0.2113 - val_accuracy: 0.9266
Epoch 4/10
809/811 [============================>.] - ETA: 0s - loss: 0.0903 - accuracy: 0.9724
Epoch 4: v

In [9]:
for layer_name in ['conv1', 'conv2', 'conv3']:
    model.get_layer(layer_name).trainable = False


In [10]:
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


In [11]:
history2 = model.fit(
    x_train, y_train,
    initial_epoch=10,
    epochs=30,
    batch_size=32,
    validation_data=(x_val, y_val),
    callbacks=[checkpoint_cb]
)


Epoch 11/30
806/811 [============================>.] - ETA: 0s - loss: 0.0391 - accuracy: 0.9881
Epoch 11: val_loss improved from 0.04608 to 0.04237, saving model to best_model.h5
811/811 [==============================] - 12s 14ms/step - loss: 0.0391 - accuracy: 0.9882 - val_loss: 0.0424 - val_accuracy: 0.9856
Epoch 12/30
805/811 [============================>.] - ETA: 0s - loss: 0.0382 - accuracy: 0.9887
Epoch 12: val_loss improved from 0.04237 to 0.04223, saving model to best_model.h5
811/811 [==============================] - 8s 10ms/step - loss: 0.0381 - accuracy: 0.9886 - val_loss: 0.0422 - val_accuracy: 0.9867
Epoch 13/30
811/811 [==============================] - ETA: 0s - loss: 0.0373 - accuracy: 0.9886
Epoch 13: val_loss improved from 0.04223 to 0.04109, saving model to best_model.h5
811/811 [==============================] - 8s 10ms/step - loss: 0.0373 - accuracy: 0.9886 - val_loss: 0.0411 - val_accuracy: 0.9858
Epoch 14/30
810/811 [============================>.] - ETA: 0s 